In [41]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt

In [14]:
pd.options.display.max_columns = None

In [18]:
# List of currency pairs
currency_pairs = ["AUDUSD=X", "NZDUSD=X", "USDBRL=X", "USDCAD=X", "USDNOK=X", "USDZAR=X"]

# Start & End Date
start_date = '2013-01-01'
end_date = '2023-01-01'

# Function to fetch exchange rate
def get_exchange_rate(currency_pairs ,start, end):
    price = yf.download(currency_pairs, start=start, end=end, group_by='ticker')
    return price

In [19]:
FX_df = get_exchange_rate(currency_pairs, start_date, end_date)

# Flatten MultiIndex columns
FX_df.columns = ['_'.join(col).strip() for col in FX_df.columns.values]

[*********************100%***********************]  6 of 6 completed


In [21]:
FX_df.head()

,USDCAD=X_Open,USDCAD=X_High,USDCAD=X_Low,USDCAD=X_Close,USDCAD=X_Adj Close,USDCAD=X_Volume,USDNOK=X_Open,USDNOK=X_High,USDNOK=X_Low,USDNOK=X_Close,USDNOK=X_Adj Close,USDNOK=X_Volume,NZDUSD=X_Open,NZDUSD=X_High,NZDUSD=X_Low,NZDUSD=X_Close,NZDUSD=X_Adj Close,NZDUSD=X_Volume,AUDUSD=X_Open,AUDUSD=X_High,AUDUSD=X_Low,AUDUSD=X_Close,AUDUSD=X_Adj Close,AUDUSD=X_Volume,USDBRL=X_Open,USDBRL=X_High,USDBRL=X_Low,USDBRL=X_Close,USDBRL=X_Adj Close,USDBRL=X_Volume,USDZAR=X_Open,USDZAR=X_High,USDZAR=X_Low,USDZAR=X_Close,USDZAR=X_Adj Close,USDZAR=X_Volume
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012-12-31,0.99436,0.99680,0.99310,0.9944,0.9944,0,5.57368,5.59670,5.55365,5.57570,5.57570,0,0.823316,0.825423,0.821288,0.822639,0.822639,0,1.039242,1.041005,1.036635,1.039199,1.039199,0,2.0456,2.0481,2.0285,2.0451,2.0451,0.0,8.47150,8.50570,8.45900,8.4709,8.4709,0.0
2013-01-01,0.99390,0.99470,0.99200,0.9920,0.9920,0,5.56390,5.56410,5.53510,5.54500,5.54500,0,0.829119,0.829119,0.826378,0.829119,0.829119,0,1.038799,1.044495,1.038702,1.039901,1.039901,0,2.0507,2.0540,2.0435,2.0507,2.0507,0.0,8.44429,8.44431,8.39780,8.3978,8.3978,0.0
2013-01-02,0.99400,0.99400,0.98360,0.9939,0.9939,0,5.55650,5.55930,5.50170,5.55730,5.55730,0,0.827061,0.839913,0.826788,0.827130,0.827130,0,1.039296,1.052499,1.039285,1.039166,1.039166,0,2.0455,2.0503,2.0350,2.0456,2.0456,0.0,8.45170,8.51930,8.41260,8.4497,8.4497,0.0
2013-01-03,0.98560,0.98680,0.98430,0.9856,0.9856,0,5.54463,5.57590,5.54191,5.54453,5.54453,0,0.833611,0.835422,0.831532,0.833403,0.833403,0,1.049307,1.052787,1.047735,1.049296,1.049296,0,2.0456,2.0485,2.0282,2.0456,2.0456,0.0,8.47790,8.59383,8.46771,8.4781,8.4781,0.0
2013-01-04,0.98846,0.99221,0.98507,0.9884,0.9884,0,5.58780,5.61794,5.58010,5.58970,5.58970,0,0.826515,0.828706,0.821625,0.826446,0.826446,0,1.045096,1.046704,1.039685,1.045303,1.045303,0,2.0488,2.0488,2.0273,2.0487,2.0487,0.0,8.58780,8.66670,8.57749,8.5877,8.5877,0.0


In [ ]:
# Function to generate features/indicators
for pair in currency_pairs:
    pair_data = FX_df[[col for col in FX_df.columns if col.startswith(pair)]]
    pair_data.columns = [col.split('_')[1] for col in pair_data.columns]  # Remove ticker from column names


In [57]:
def get_indicators(pair_data):
    # Calculate RSI
    pair_data['RSI'] = ta.rsi(pair_data['Close'], length = 14)

    # Calculate ADX
    adx = ta.adx(pair_data['High'], pair_data['Low'], pair_data['Close'], length=14)
    pair_data = pair_data.join(adx)

    # Calculate SMA
    pair_data['SMA_5'] = ta.sma(pair_data['Close'], length=5)
    pair_data['SMA_8'] = ta.sma(pair_data['Close'], length=8)

    # Calculate Stochastic Oscillator
    stoch = ta.stoch(pair_data['High'], pair_data['Low'], pair_data['Close'], k=14, d=3)
    pair_data = pair_data.join(stoch)

    return pair_data


In [58]:
cleaned_data = {}
for pair in currency_pairs:
    pair_data = FX_df[[col for col in FX_df.columns if col.startswith(pair)]].copy()
    pair_data.columns = [col.split('_')[1] for col in pair_data.columns]
    cleaned_data[pair] = get_indicators(pair_data).dropna()
    

In [62]:
cleaned_data[pair]

,Open,High,Low,Close,Adj Close,Volume,ADX_14,DMP_14,DMN_14,SMA_5,SMA_8
Date,,,,,,,,,,,
2013-02-06,8.820300,8.936100,8.813200,8.820100,8.820100,0.0,38.330196,18.607143,16.026615,8.912160,8.952412
2013-02-07,8.900900,8.916500,8.863400,8.900700,8.900700,0.0,35.042941,17.478693,15.054664,8.888560,8.947100
2013-02-08,8.912120,8.943000,8.870000,8.911100,8.911100,0.0,32.702861,18.365728,14.345188,8.879160,8.924175
2013-02-11,8.872100,8.933620,8.864600,8.872100,8.872100,0.0,30.540376,17.524801,14.046567,8.883040,8.905587
2013-02-12,8.907400,8.988020,8.903410,8.907920,8.907920,0.0,29.592917,19.773666,12.972250,8.882384,8.891740
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,16.959499,17.043699,16.711201,16.959499,16.959499,0.0,19.774226,16.922904,18.950979,17.179940,17.266249
2022-12-27,17.020491,17.313810,16.887600,17.020491,17.020491,0.0,18.964881,20.646856,17.431727,17.117498,17.248924
2022-12-28,17.193501,17.282301,17.094801,17.193501,17.193501,0.0,18.213347,19.607038,16.553829,17.094638,17.210737


# Backtesting

In [64]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import talib

In [182]:
# Define strategy
class SMACrossADXStrategy(Strategy):
    def init(self):
        self.sma5 = self.I(talib.SMA, self.data.Close, 5)
        self.sma8 = self.I(talib.SMA, self.data.Close, 21)
        self.adx = self.I(talib.ADX, self.data.High, self.data.Low, self.data.Close, 14)
        self.rsi = self.I(talib.RSI, self.data.Close, 14)
        self.position_opened = False  # Flag to track if a position is already open

    def next(self):
        if not self.position_opened:
            if crossover(self.sma5, self.sma8) and self.adx[-1] > 25:
                self.buy()
                self.position_opened = True
            elif crossover(self.sma8, self.sma5) and self.adx[-1] > 25:
                self.sell()
                self.position_opened = True
        else:
            if self.position.is_long and crossover(60,self.rsi):
                self.position.close()
                self.position_opened = False
            elif self.position.is_short and crossover(self.rsi,40):
                self.position.close()
                self.position_opened = False


# Run backtest
bt = Backtest(cleaned_data[pair], SMACrossADXStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()
print(stats)

c:\Users\Bangkit\anaconda3\envs\qts\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Bangkit\anaconda3\envs\qts\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Bangkit\anaconda3\envs\qts\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\Bangkit\anaconda3\envs\qts\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2013-02-06 00:00:00
End                       2022-12-30 00:00:00
Duration                   3614 days 00:00:00
Exposure Time [%]                   17.269545
Equity Final [$]                 10735.873593
Equity Peak [$]                  10928.589594
Return [%]                           7.358736
Buy & Hold Return [%]               91.841376
Return (Ann.) [%]                    0.698399
Volatility (Ann.) [%]                6.942578
Sharpe Ratio                         0.100597
Sortino Ratio                        0.135015
Calmar Ratio                         0.029612
Max. Drawdown [%]                  -23.584842
Avg. Drawdown [%]                   -4.183583
Max. Drawdown Duration     2502 days 00:00:00
Avg. Drawdown Duration      323 days 00:00:00
# Trades                                   23
Win Rate [%]                        69.565217
Best Trade [%]                         6.7746
Worst Trade [%]                    -14.090103
Avg. Trade [%]                    